## Dependencies and Dataset

In [ ]:
!pip install -U datasets
!pip install evaluate

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
try:
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')

    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text'])
    dataset_tts_train = dataset_tts_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text'])
    dataset_tts_test  = dataset_tts_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
    dataset_tts_val   = dataset_tts_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

    # `interleave_datasets` used to merge training, test, val from three sources
    # Just to ensure the datasets are mix of examples
    dataset = DatasetDict({ 'train': interleave_datasets([dataset_scc_train,
                                                          dataset_tts_train,
                                                          dataset_ks_train]),
                            'test': interleave_datasets([dataset_scc_test,
                                                         dataset_tts_test,
                                                         dataset_ks_test]),
                            'validation': interleave_datasets([dataset_scc_val,
                                                               dataset_tts_val,
                                                               dataset_ks_val])})

    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset")

dataset

Loaded Merged Dataset


DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14838
    })
})

In [ ]:
dataset['train'][0]

{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)'}

## Data Preprocessing

### Convert the datasets into explicit instructions for the LLM, then preprocess the prompt-response dataset into tokens, and pull out their input_ids

In [ ]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to('cuda')

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def tokenize_function(example):
  # print(len(example['question']))
  start_prompt="Tables:\n"
  middle_prompt='\n\nQuestion:\n'
  end_prompt='\n\nAnswer:\n'

  data_zip = zip(example['context'], example['question'])
  prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
  example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
  example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
  # print(prompt[0])
  # print()

  return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

try:
  tokenized_datasets = load_from_disk("tokenized_datasets")
  print("Loaded Tokenized Dataset")
except:
  tokenized_datasets = dataset.map(tokenize_function, batched=True)
  tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer'])

  tokenized_datasets.save_to_disk("tokenized_datasets")
  print("Tokenized and Saved Dataset")

Loaded Tokenized Dataset


In [ ]:
print(tokenized_datasets.keys())
print(tokenized_datasets['train'][0].keys())
print(tokenized_datasets['train'][0]['input_ids'][:10])
print(tokenized_datasets['train'][0]['labels'][:10])
print(tokenized_datasets)

dict_keys(['train', 'test', 'validation'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]
[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (118695, 2)
Validation: (14838, 2)
Test: (14835, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


## Test Base Model

In [ ]:
index = 3

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_49 (date VARCHAR, home_team VARCHAR)

Question:
When the Home team was north melbourne what was the Date of the game?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT date FROM table_name_49 WHERE home_team = "north melbourne"

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Question


**OMG, what is this, not even output an answer.**

## Perform Fine-Tuning with LoRA
- Since Full Fine-Tuning might use up Colab's runtime, memory, we changed to Fine-Tuning using QLoRA

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes  # we need latest transformers for this

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-gu7bslc9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gu7bslc9
  Resolved https://github.com/huggingface/transformers.git to commit 9f51dc25357bcde280a02b59e80b66248b018ca4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10771428 sha256=837ddfc04798905fbeb9c1de81c673b43aaacc257b6af6cbe400d8a81e9702a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-y13ptspa/wheels/bb/cb/6e/60d1799a10be9a6eeb01cb85fd51411adbfd9bc95d885ac15e
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
%%time

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
import time
from datasets import load_dataset
import bitsandbytes as bnb

try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(
        "finetuned_model_qlora_3_epoch",
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    to_train = False

except:
    to_train = True
    # Load the base model with 4-bit quantization
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare the model for QLoRA training
    finetuned_model = prepare_model_for_kbit_training(
        finetuned_model,
        use_gradient_checkpointing=True
    )

    # Configure LoRA
    lora_config = LoraConfig(
        r=16,                     # Rank of update matrices
        lora_alpha=32,           # Alpha parameter for LoRA scaling
        target_modules=["q", "v"],  # Which modules to apply LoRA to
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_2_SEQ_LM"
    )

    # Add LoRA adapters to the model
    finetuned_model = get_peft_model(finetuned_model, lora_config)

if to_train:
    output_dir = f'./sql-training-qlora-{str(int(time.time()))}'

    # Modified training arguments for QLoRA
    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-4,           # Lower learning rate for LoRA
        num_train_epochs=3,
        per_device_train_batch_size=8,  # Reduced batch size
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',
        eval_steps=500,
        save_strategy="steps",
        save_steps=500,
        warmup_steps=100,
        fp16=True,                    # Use mixed precision training
        optim="paged_adamw_32bit"     # Use optimized AdamW optimizer
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
    )

    trainer.train()

    # Save the trained model
    finetuned_model.save_pretrained("finetuned_model_qlora_3_epoch")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mahkotasteam (mahkotasteam-asia-pacific-university-of-technology-innov) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Full Fine-Tuning

**2 Epochs**
**5e-3**
Time Taken = 2h 49m 1s on a laptop with a GeForce RTX 3070 GPU

Training Loss = 0.023100
Validation Loss = 0.013285

In [ ]:
# try:
#     finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
#     finetuned_model = finetuned_model.to('cuda')
#     to_train = False

# except:
#     to_train = True
#     finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
#     finetuned_model = finetuned_model.to('cuda')
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# %%time

# if to_train:
#     output_dir = f'./sql-training-{str(int(time.time()))}'

#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         learning_rate=5e-3,
#         num_train_epochs=2,
#         per_device_train_batch_size=16,     # batch size per device during training
#         per_device_eval_batch_size=16,      # batch size for evaluation
#         weight_decay=0.01,
#         logging_steps=50,
#         evaluation_strategy='steps',        # evaluation strategy to adopt during training
#         eval_steps=500,                     # number of steps between evaluation
#     )

#     trainer = Trainer(
#         model=finetuned_model,
#         args=training_args,
#         train_dataset=tokenized_datasets['train'],
#         eval_dataset=tokenized_datasets['validation'],
#     )

#     trainer.train()

#     finetuned_model.save_pretrained("finetuned_model_2_epoch")

## Test the Fine-Tuning Using QLoRA with Zero Short Inferencing

In [ ]:
index = 0
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

## Evaluate the Model Quantitatively (Using ROUGE Metric)

In [ ]:
# Perform inferences for test dataset. Do 25 only, due to time it takes.

questions = dataset['test'][0:25]['question']
contexts = dataset['test'][0:25]['context']
human_baseline_answers = dataset['test'][0:25]['answer']

original_model_answers = []
finetuned_model_answers = []

for idx, question in enumerate(questions):

    prompt = f"""Tables:
{contexts[idx]}

Question:
{question}

Answer:
"""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    human_baseline_text_output = human_baseline_answers[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_answers.append(original_model_text_output)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_text_output)

zipped_summaries = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_answers', 'original_model_answers', 'finetuned_model_answers'])
# df

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_answers,
    references=human_baseline_answers[0:len(original_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)


finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=human_baseline_answers[0:len(finetuned_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results)

ORIGINAL MODEL:

{'rouge1': 0.031970284742291306, 'rouge2': 0.005, 'rougeL': 0.03070044347245003, 'rougeLsum': 0.03121247624254732}
FINE-TUNED MODEL:

{'rouge1': 0.923359923692127, 'rouge2': 0.8863291968739871, 'rougeL': 0.9176464597549342, 'rougeLsum': 0.9182149521321223}

**The Results from Full Fine-Tuning and Evaluation Metrics is not trained by me, but following an article from [[Medium]](https://medium.com/@martinkeywood/fine-tuning-a-t5-small-model-to-generate-sql-from-natural-language-with-92-3-accuracy-fb29e062c638), can check for his [[GitHub]](https://gist.github.com/mkeywood1/9e8411aef44cf18009aa3e4776501c08) as well.**